In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
sns.set(style="ticks")
plt.style.use('seaborn-whitegrid')

rules = pd.read_pickle('final_dataset.pkl')
data = pd.read_csv('ga_dump_for_lhl.csv', sep = ';')
insights = pd.read_csv('final_insights.csv')

In [22]:
insights.drop('Unnamed: 0', axis = 1, inplace = True)

# **Deriving Business Insights Using Google Analytics Data**

### Analyzing client data to derive insights from customer behaviour and identify areas of improvement using data science techniques
### Presented by: Safwaan Hassan

 

## Introduction to Business Problem

## Client
* Large sportswear retail website based out of Germany

## Data
* Provided Google Analytics data containing information on a user session level
    * Data contained information on how a user navigated the website and what products they viewed/purchased in a given session

## Overview of the Provided Dataset
Note - This is a cleaned version of the raw dataset provided

In [13]:
cols = data.columns.tolist()
clean_cols = [col.strip('ga:') for col in cols]
data.columns = clean_cols

def drop_cols(col):
  drop_cols = []
  for i in col:
    if len(data[i].value_counts()) == 1:
      drop_cols.append(i)
  
  return drop_cols

cols_to_drop = drop_cols(data.columns)
data.drop(cols_to_drop, axis = 1, inplace = True)

data['productName'] = data['productName'].str.lstrip()
data['productName'] = data['productName'].str.replace("  ", " ")

dateHourMinute = pd.to_datetime(data.dateHourMinute.values, format = "%Y%m%d%H%M").tolist()
data['dateHourMinute'] = dateHourMinute

data.rename(columns={'dimension3': 'sessionID'}, inplace = True)

In [14]:
data.head()

,pagePath,previousPagePath,landingPagePath,exitPagePath,dateHourMinute,sessionID,productName,productDetailViews,productListViews,itemRevenue,productListClicks
0,/,(entrance),/order/succes?oid=2000075871&paymentId=PAYID-L...,/order/succes?oid=2000075871&paymentId=PAYID-L...,2020-04-03 16:51:00,1585925484894.5r1t58lo,Laufschuhe Nike W EPIC REACT FLYKNIT 2,0,0,122.201307,0
1,/,(entrance),/order/succes?oid=2000083281&paymentId=PAYID-L...,/c/ausverkauf-schuhe/gender-1,2020-04-13 00:03:00,1586729138631.ir91604q,Schuhe Nike AIR MAX LTD 3,0,0,146.668754,0
2,/,(entrance),/order/succes?oid=2000083481&paymentId=PAYID-L...,/user/orderdetail/2000083481,2020-04-13 11:41:00,1586770910477.5blqswhq,Hose Nike W NK SPEED TGHT 7_8 MATTE,0,0,74.625715,0
3,/,(entrance),/order/succes?oid=2000083481&paymentId=PAYID-L...,/user/orderdetail/2000083481,2020-04-13 11:41:00,1586770910477.5blqswhq,Kapuzenjacke Nike W NSW WR JKT,0,0,75.985017,0
4,/,(entrance),/order/succes?oid=2000083481&paymentId=PAYID-L...,/user/orderdetail/2000083481,2020-04-13 11:41:00,1586770910477.5blqswhq,Langarm-T-Shirt Nike W NK MIDLAYER QZ SWSH RUN,0,0,46.080359,0


In [15]:
data.shape # Number of rows and columns provided

(319865, 11)

## Driving Motivations for Analysis
### Customer Behaviour
* **Products that were searched and purchased most frequently**
* **Grouping products with others that were viewed by similar users**
* **Analyzing how users navigated website by way of site path analysis**
* **User interaction to purchase conversion**

### Product Analysis
* **Which products brought most traffic to website**
* **Client pricing versus competition pricing**

## Analysis - Most Searched Items

<img src = 'imgs/top_30_s.png'>

## Analysis - Most Purchased Items

<img src = 'imgs/top_30_purch.png'>

## Analysis - Website Traffic

<img src = 'imgs/time_of_day.png'>

## Analysis - Website Traffic
<img src = 'imgs/day_of_week.png'>

## Analysis - Website Traffic

<img src = 'imgs/hour,day.png'>

## Diving Into the Data: Identifying Areas of Improvement

* Using path information and revenue details, user sessions could be further investigated for what items they searched and what items were eventually purchased
* From this, the following was observed

In [16]:
data['search'] = 0
data.loc[(data['pagePath'].str.contains('/s/') == True) | (data['previousPagePath'].str.contains('/s/') == True) | (data['landingPagePath'].str.contains('/s') == True) | (data['exitPagePath'].str.contains('/s/') == True) | (data['itemRevenue'] <= 0), 'search'] = 1
data['purchased'] = 0
data.loc[data['itemRevenue'] > 0, 'purchased'] = 1

searches = data.loc[(data['search'] == 1) & (data['purchased'] == 0)]
purchases = data.loc[data['purchased'] == 1]


def basket(dataframe, param):
  basket = dataframe.groupby(['sessionID', 'productName'])[param].sum().unstack().reset_index().fillna(0).set_index('sessionID')
  return basket


search_basket = basket(searches, 'search')
purchase_basket = basket(purchases, 'purchased')

purchased_items = []
num_purchases = []
cust_id = []
for i in purchase_basket.index:
  query = purchase_basket.loc[i]
  items = query.loc[query > 0].index[:-1].tolist()
  num_purchases.append(len(items))
  purchased_items.append(items)
  cust_id.append(i)

purchases_df = pd.DataFrame()
purchases_df['sessionID'] = cust_id
purchases_df['purchases'] = purchased_items
purchases_df['num_purchases'] = num_purchases

searched_items = []
num_searches = []
cust_id = []
for i in search_basket.index:
  query = search_basket.loc[i]
  items = query.loc[query > 0].index[:-1].tolist()
  num_searches.append(len(items))
  searched_items.append(items)
  cust_id.append(i)

searches_df = pd.DataFrame()
searches_df['sessionID'] = cust_id
searches_df['searches'] = searched_items
searches_df['num_searches'] = num_searches

cust_list = pd.DataFrame()
cust_list['sessionID'] = data.sessionID.unique() 

cust_list = pd.merge(cust_list, searches_df, on='sessionID', how='left')
cust_list = pd.merge(cust_list, purchases_df, on = 'sessionID', how='left')

cust_list['searches'].fillna('-', inplace = True)
cust_list['num_searches'].fillna(0, inplace = True)
cust_list['num_purchases'].fillna(0, inplace = True)
cust_list['purchases'].fillna('-', inplace = True)

In [17]:
no_purchase_search = cust_list.loc[(cust_list['num_purchases'] == 0) & (cust_list['num_searches'] > 0.0)]['num_searches'].sum()
print("Number of searches that led to no purchases: {}".format(no_purchase_search))
print("Percentage of searches that do not lead to purchases: {}".format(no_purchase_search / cust_list.loc[(cust_list['num_searches'] > 0)]['num_searches'].sum() * 100))

Number of searches that led to no purchases: 73104.0
Percentage of searches that do not lead to purchases: 95.87032641338702


**Evidence suggests that there is room for improvement in converting searches to purchases**

The rest of this presentation outlines my proposed approaches to solving this problem

## Insight Into User Behaviour

The following table details my findings for the 20 most purchased items in the period of analysis

My reasons for picking the 20 best sellers:

In [29]:
total_rev = data['itemRevenue'].sum()
top20_rev = insights['itemRevenue'].sum()

print("Top 20 best sellers account for {}% of revenue in period of analysis".format(round((top20_rev / total_rev) * 100, 3)))
print("Top 20 revenue: {:,}".format(round(top20_rev, 2)))

Top 20 best sellers account for 44.797% of revenue in period of analysis
Top 20 revenue: 505,346.71


## Insight Into User Behaviour

In [23]:
insights.head(2)

,productName,altName,num_purchases,productDetailViews,productListViews,productListClicks,path_search,listClicks/listViews,detailViews/listViews,purchase/listView,purchase/detailViews,purchase/listClicks,itemRevenue,avgPrice,minPrice,minSeller,maxPrice,maxSeller,clientAveragePrice
0,ZOOMX VAPORFLY NEXT%,Laufschuhe Nike ZOOMX VAPORFLY NEXT%,221.0,1146,15738.0,2097.0,9,0.133244,0.072817,0.014042,0.192845,0.105389,86318.302944,305.9825,274.95,sportler.com,399.00,StockX,274.9
1,ZOOM PEGASUS TURBO 2,Laufschuhe Nike ZOOM PEGASUS TURBO 2,229.0,433,5734.0,819.0,6,0.142832,0.075514,0.039937,0.528868,0.279609,48844.202678,126.8300,84.90,Zalando,159.95,Sport Klingenmaier,145.0


The ratios are explained as follows:
* **listClicks/listViews** - How often the product was inspected when shown in a list
* **detailViews/listViews** - How often the product details were looked at when shown in a list
* **purchase/listView** - How often a product was purchased when viewed in a list
* **purchase/detailViews** - How often a product was purchased when details were looked at
* **purchase/listClicks** - How often a product was purchased when inspected in a list
    

In [27]:
insights['clientPrice/avgPrice'] = insights['clientAveragePrice'] / insights['avgPrice']

## Insight Into User Behaviour

The data shows that there is an average conversion rate of **58%** from Detail Views to Purchases

However, on average, only **12%** of List Views are converted to Detail Views

This suggests that the products recommended to users when they use the search function are not optimized to that specific user or the product that was originally searched

## Insight Into User Behaviour


This area can be improved upon by reconsidering the recommender engine currently in use
* Currently, the search function implemented on the website recommends products that match the search in terms of **keyword relevancy**

* This is reflected by the top 5 most purchased products, which have similar and intersecting names

In [28]:
insights.head().productName

0       ZOOMX VAPORFLY NEXT%
1       ZOOM PEGASUS TURBO 2
2     AIR VAPORMAX FLYKNIT 3
3    AIR ZOOM ALPHAFLY NEXT%
4                 ZOOM FLY 3
Name: productName, dtype: object

## Building a Better Recommender

To improve on the current model, a search engine based on machine learning may yield better results

* The one shown in this demo was built using the **Apriori** algorithm
* The algorithm identifies frequent item sets within a database of transactions
* In laymans terms, if item A is bought, how often are other items within the database also bought
* In this application, the algorithm was used to find the relationships between the products different users interacted with across all the recorded sessions in the database

When applied to the provided dataset, the algorithm uncovered over 3,000,000 association rules. The following is a sample of these results:

In [47]:
rules.head()

,antecedents,consequents
6666,ZOOMX VAPORFLY NEXT%,"ZOOM FLY 3,REACT INFINITY RUN FK"
6662,"ZOOM FLY 3,ZOOMX VAPORFLY NEXT%",REACT INFINITY RUN FK
6663,"ZOOM FLY 3,REACT INFINITY RUN FK",ZOOMX VAPORFLY NEXT%
6664,"ZOOMX VAPORFLY NEXT%,REACT INFINITY RUN FK",ZOOM FLY 3
6665,ZOOM FLY 3,"ZOOMX VAPORFLY NEXT%,REACT INFINITY RUN FK"


## Building a Better Recommender

Using these results, we can better identify what products are most similar to others and in turn, provide better suggestions

The following are examples of how this engine works

In [29]:
def recommendation(item, num_suggestions = 10):
  
  item = item.upper()

  set1 = rules.loc[rules['antecedents'].str.contains(item) == True]['antecedents']
  set2 = rules.loc[rules['antecedents'].str.contains(item) == True]['consequents']

  set1 = (i.split(',') for i in set1)
  set1 = set(x for l in set1 for x in l)

  set2 = (i.split(',') for i in set2)
  set2 = set(x for l in set2 for x in l)
  combined = set1.union(set2)


  for i, val in enumerate(itertools.islice(combined, num_suggestions)):
    print(val)
  
  return

In [30]:
recommendation('zoom', num_suggestions = 5)

Cloudboom
WMNS ZM PEG 36 TRAIL GTX
AIR ZOOM PEGASUS 37
REACT INFINITY RUN FK
W PEGASUS TRAIL 2


In [31]:
recommendation('ultraboost', num_suggestions = 5)

SL20 W
SL20
ULTRABOOST PB W
UA HOVR Machina
GEL-DS TRAINER 25 W


## Understanding the Competition

* Competition pricing was also analyzed as part of this project
* Prices for each of the 20 best sellers was scraped from Google searches to find how client competitors priced each of the products

In [32]:
insights[insights['detailViews/listViews']>0].sort_values('detailViews/listViews').head()

,productName,altName,num_purchases,productDetailViews,productListViews,productListClicks,path_search,listClicks/listViews,detailViews/listViews,purchase/listView,purchase/detailViews,purchase/listClicks,itemRevenue,avgPrice,minPrice,minSeller,maxPrice,maxSeller,clientAveragePrice,clientPrice/avgPrice
9,ZOOM SUPERFLY ELITE,Spikes Nike ZOOM SUPERFLY ELITE,84.0,153,8488.0,408.0,4,0.048068,0.018025,0.009896,0.549020,0.205882,15532.615271,125.128333,86.99,shop4runners,57.50,Top4Running,113.9,0.910265
4,ZOOM FLY 3,Laufschuhe Nike ZOOM FLY 3,239.0,425,22211.0,1078.0,8,0.048535,0.019135,0.010760,0.562353,0.221707,47208.064961,135.042500,101.00,StockX,159.90,Top4Running,159.9,1.184072
5,REACT INFINITY RUN FK,Laufschuhe Nike REACT INFINITY RUN FK,221.0,340,13338.0,862.0,10,0.064627,0.025491,0.016569,0.650000,0.256381,41456.038903,116.140000,91.90,Jogging-Point.de,154.95,Zalando,121.5,1.046151
6,ZOOM VAPORFLY 4% FLYKNIT,Laufschuhe Nike ZOOM VAPORFLY 4% FLYKNIT,91.0,520,12691.0,2563.0,5,0.201954,0.040974,0.007170,0.175000,0.035505,26938.417729,207.918000,70.00,eBay.de,159.99,Zalando,189.9,0.913341
0,ZOOMX VAPORFLY NEXT%,Laufschuhe Nike ZOOMX VAPORFLY NEXT%,221.0,1146,15738.0,2097.0,9,0.133244,0.072817,0.014042,0.192845,0.105389,86318.302944,305.982500,274.95,sportler.com,399.00,StockX,274.9,0.898417


## Understanding the Competition


* The data shows that the products with the lowest purchase conversions were the ones that were sold at higher than market average prices


* Due to increasing popularity of resale markets like StockX, average market prices for shoes are more volatile than ever


* Those with high demand sell for a much higher valuation than retail, and the opposite holds true


* This volatility has not yet been addressed by large scale retailers, although the data suggests its effects are being felt


* While a slightly more difficult and bureucratic solution to implement, applying an algorithm that dynamically adjusted website prices as average price on the market changed could improve search to purchase conversion

## Closing Comments

* An improvement on this project would be to make it more dynamic
    * Online scraping of competition data, updating recommender engine with live searches
* Existing recommender would need to be sped up before it is deployed
    * Currently processing at 3 seconds per request, far too slow for full scale deployment (28,800 requests per day)